## 2.1

### Describe this calculation using Python

Q: How many apples and pears did a person bought?

- apple @120
- pear @150
- total quantity 10
- total price 1440

A:

Put `x` as number of apples and `y` as of pears.

- 120 * x + 150 * y = 1440
- x + y = 10

In result, x = 2 and y = 8

In [1]:
import pulp

problem = pulp.LpProblem("SLE", pulp.LpMaximize)

In [2]:
x = pulp.LpVariable('x', cat=pulp.LpContinuous)
y = pulp.LpVariable('y', cat=pulp.LpContinuous)

In [3]:
problem += 120 * x + 150 * y == 1440
problem += x + y == 10

In [4]:
status = problem.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/5b7b69c221f74006b58d253d00eb7231-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/5b7b69c221f74006b58d253d00eb7231-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 13 RHS
At line 16 BOUNDS
At line 20 ENDATA
Problem MODEL has 2 rows, 3 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-2) rows, 0 (-3) columns and 0 (-4) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value -0
After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00
Option

'Optimal'

In [5]:
x.value(), y.value()

(2.0, 8.0)

## 2.2

Q: How can we maximize our profit under these condition?

- Product *p* and *q* require material *m* and *n*.
- 1kg of *p* consists of
    - 1kg of *m*
    - 2kg og *n*
- 1kg of *q* consists of
    - 3kg of *m*
    - 1kg of *n*
- The stock of *m* is 30kg
- The stock of *n* is 40kg
- The profit per 1kg of *p* is 10k and of *q* is 20k


A:

Put x as total weight of produced *p* and y as of *q*

- x + 3y <= 30
- 2x + y <= 40
- x >= 0
- y >= 0
- Maximize p * 10_000 + q * 20_000

In result, x = 18 and y = 4

In [6]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)

x = pulp.LpVariable('x', cat=pulp.LpContinuous)
y = pulp.LpVariable('y', cat=pulp.LpContinuous)

In [7]:
problem += 1 * x + 3 * y <= 30
problem += 2 * x + 1 * y <= 40
problem += x >= 0
problem += y >= 0
problem += x + 2 * y  # objective function to maximize

In [8]:
status = problem.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/79e48410f1544b8abc9252c24fbabf74-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/79e48410f1544b8abc9252c24fbabf74-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 18 RHS
At line 23 BOUNDS
At line 26 ENDATA
Problem MODEL has 4 rows, 2 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-2) rows, 2 (0) columns and 4 (-2) elements
0  Obj -0 Dual inf 2.999998 (2)
2  Obj 26
Optimal - objective value 26
After Postsolve, objective 26, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 26 - 2 iterations time 0.002, Presolve 0.00
Option for p

'Optimal'

In [9]:
x.value(), y.value(), problem.objective.value()

(18.0, 4.0, 26.0)

## 2.3

Q: How can we maximize our profit under these condition?

- Products: p1 p2 p3 p4
- Materials: m1 m2 m3
- Material requirements: see csv file
- Stocks: see csv file
- Profits: see csv file

### read resources

In [10]:
import pandas as pd

In [11]:
stocks = pd.read_csv('./resources/2.tutorial/stocks.csv')
stocks

,m,stock
0,m1,35
1,m2,22
2,m3,27


In [12]:
requires = pd.read_csv('./resources/2.tutorial/requires.csv')
requires

,p,m,require
0,p1,m1,2
1,p1,m2,0
2,p1,m3,1
3,p2,m1,3
4,p2,m2,2
5,p2,m3,0
6,p3,m1,0
7,p3,m2,2
8,p3,m3,2
9,p4,m1,2


In [13]:
gains = pd.read_csv('./resources/2.tutorial/gains.csv')
gains

,p,gain
0,p1,3
1,p2,4
2,p3,4
3,p4,5


In [14]:
P = gains['p'].tolist()
P

['p1', 'p2', 'p3', 'p4']

In [15]:
M = stocks['m'].tolist()
M

['m1', 'm2', 'm3']

### define constants

- stock_m => m_1: x, m_2: y, ...
- require_p_m =>
    - r_1_1: a, r_1_2: b, ...
    - r_2_1: c, r_2_2: d, ...
    - ...
- gain_p => g_1: x, g_2: y, ...

In [16]:
stock = {row.m: row.stock for row in stocks.itertuples()}
stock

{'m1': 35, 'm2': 22, 'm3': 27}

In [17]:
require = requires.set_index(['p', 'm']).to_dict()['require']
require

{('p1', 'm1'): 2,
 ('p1', 'm2'): 0,
 ('p1', 'm3'): 1,
 ('p2', 'm1'): 3,
 ('p2', 'm2'): 2,
 ('p2', 'm3'): 0,
 ('p3', 'm1'): 0,
 ('p3', 'm2'): 2,
 ('p3', 'm3'): 2,
 ('p4', 'm1'): 2,
 ('p4', 'm2'): 2,
 ('p4', 'm3'): 2}

In [18]:
gain = dict(zip(gains['p'], gains['gain']))
gain

{'p1': 3, 'p2': 4, 'p3': 4, 'p4': 5}

### define as LP


In [19]:
problem = pulp.LpProblem('LP2.3', pulp.LpMaximize)

X_p = pulp.LpVariable.dicts('x', P, cat=pulp.LpContinuous)

#### define constraints

- X_p >= 0
    - =>
        - X_(p_1) >= 0
        - X_(p_2) >= 0
        - X_(p_3) >= 0
        - X_(p_4) >= 0
- sum(require_p_m * X_p) <= stock_m
    - =>
        - require_(p_1)_(m_1) * x_(p_1) + require_(p_2)_(m_1) * x_(p_2) + require_(p_3)_(m_1) * x_(p_3) + require_(p_4)_(m_1) * x_(p_4) <= stock_(m_1)
        - require_(p_1)_(m_2) * x_(p_1) + require_(p_2)_(m_2) * x_(p_2) + require_(p_3)_(m_2) * x_(p_3) + require_(p_4)_(m_2) * x_(p_4) <= stock_(m_2)
        - require_(p_1)_(m_3) * x_(p_1) + require_(p_2)_(m_3) * x_(p_2) + require_(p_3)_(m_3) * x_(p_3) + require_(p_4)_(m_3) * x_(p_4) <= stock_(m_3)


In [20]:
for p in P:
    problem += X_p[p] >= 0

In [21]:
for m in M:
    problem += pulp.lpSum([require[p,m] * X_p[p] for p in P]) <= stock[m]

#### define objective function

In [22]:
problem += pulp.lpSum([gain[p] * X_p[p] for p in P])

In [23]:
status = problem.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/ae435ab26b99474eb244f420d6e2dcd1-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/ae435ab26b99474eb244f420d6e2dcd1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 30 RHS
At line 38 BOUNDS
At line 43 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-4) rows, 4 (0) columns and 9 (-4) elements
0  Obj -0 Dual inf 17.499996 (4)
4  Obj 80.428571
Optimal - objective value 80.428571
After Postsolve, objective 80.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 80.42857143 - 4 iterations time 

'Optimal'

In [24]:
[X_p[p].value() for p in P], problem.objective.value()

([12.142857, 3.5714286, 7.4285714, 0.0], 80.42857099999999)

### Change to Integer Programming

In [25]:
ip = pulp.LpProblem('IP2.3', pulp.LpMaximize)
X_p = pulp.LpVariable.dicts('x', P, cat=pulp.LpInteger)

for p in P:
    ip += X_p[p] >= 0

for m in M:
    ip += pulp.lpSum([require[p,m] * X_p[p] for p in P]) <= stock[m]

ip += pulp.lpSum([gain[p] * X_p[p] for p in P])

In [26]:
status = ip.solve()
pulp.LpStatus[status]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ta.nakamura/src/github.com/na9amura/try-or-tools/venv/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/1ee42c68664a48b48117419669b20273-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/y3/x37rtt0s6fn4r9r7__s24rzc0000gp/T/1ee42c68664a48b48117419669b20273-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 38 RHS
At line 46 BOUNDS
At line 51 ENDATA
Problem MODEL has 7 rows, 4 columns and 13 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 80.4286 - 0.00 seconds
Cgl0004I processed model has 3 rows, 4 columns (4 integer (0 of which binary)) and 9 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -76 found by DiveCoefficient after 0 iterations a

'Optimal'

In [27]:
[X_p[p].value() for p in P], ip.objective.value()

([13.0, 3.0, 7.0, -0.0], 79.0)